In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import psycopg2
import json

In [24]:
electric_car_count = []
response = requests.get("https://chargeinfo.ksga.org/ws/statistics/ev/list",
                        headers= {
                                'user-agent':'Mozilla 5.0',
                                'Referer':'https://chargeinfo.ksga.org/front/statistics/evCar' ,
                                        })
result = json.loads(response.text)
for year_dict in result.get('result'):
    electric_car_count.append(list(year_dict.values()))

In [25]:
electric_car_last_count = []
electric_car_last_count.append(electric_car_count[0])
for i in electric_car_count: 
    if i[0][-2:]=='12':
        electric_car_last_count.append(i)
    

In [26]:
electric_car_last_count

[['202402',
  72981,
  35022,
  30414,
  41189,
  12571,
  18003,
  7888,
  4475,
  115414,
  18240,
  20009,
  24055,
  19804,
  24336,
  26985,
  36534,
  39535],
 ['202312',
  72937,
  34643,
  30396,
  40397,
  12538,
  17889,
  7838,
  4393,
  114117,
  18236,
  19972,
  24130,
  19795,
  24200,
  26776,
  36225,
  39418],
 ['202212',
  59327,
  22063,
  24161,
  26242,
  9096,
  14476,
  5061,
  3034,
  77648,
  14012,
  15140,
  16611,
  12727,
  15387,
  19154,
  22740,
  32976],
 ['202112',
  40564,
  12375,
  16185,
  12820,
  5194,
  7701,
  3166,
  1859,
  39958,
  7946,
  8194,
  9991,
  7365,
  8708,
  11240,
  12606,
  25571],
 ['202012',
  23393,
  5355,
  12630,
  5366,
  3210,
  4469,
  2274,
  1148,
  20477,
  4078,
  3883,
  5489,
  3323,
  5223,
  7051,
  6308,
  21285],
 ['201912',
  14952,
  3216,
  11313,
  2598,
  2464,
  2555,
  1447,
  903,
  11750,
  2445,
  2412,
  2841,
  1841,
  3326,
  4051,
  3626,
  18178],
 ['201812',
  9564,
  1567,
  6605,
  1284,
 

In [28]:
pd.DataFrame(electric_car_last_count)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,202402,72981,35022,30414,41189,12571,18003,7888,4475,115414,18240,20009,24055,19804,24336,26985,36534,39535
1,202312,72937,34643,30396,40397,12538,17889,7838,4393,114117,18236,19972,24130,19795,24200,26776,36225,39418
2,202212,59327,22063,24161,26242,9096,14476,5061,3034,77648,14012,15140,16611,12727,15387,19154,22740,32976
3,202112,40564,12375,16185,12820,5194,7701,3166,1859,39958,7946,8194,9991,7365,8708,11240,12606,25571
4,202012,23393,5355,12630,5366,3210,4469,2274,1148,20477,4078,3883,5489,3323,5223,7051,6308,21285
5,201912,14952,3216,11313,2598,2464,2555,1447,903,11750,2445,2412,2841,1841,3326,4051,3626,18178
6,201812,9564,1567,6605,1284,1447,1334,847,394,6383,1377,1199,1127,997,1974,2001,2107,15549
7,201712,4797,816,2005,542,548,303,356,95,2290,459,281,336,336,960,756,1022,9206
8,201612,1498,366,344,207,239,74,103,25,650,150,60,170,57,446,278,559,5629
9,201512,1151,232,88,129,186,29,54,10,313,81,28,140,37,290,147,428,2369


In [21]:
# dbeaver 연결

conn = psycopg2.connect(
    host = 'localhost',
    dbname = 'postgres',
    user = 'postgres',
    password = '0000',
    port = 5431
) 

In [22]:
with conn.cursor() as cur: 
    # years 테이블 생성
    cur.execute('''
        CREATE TABLE IF NOT EXISTS years (
            year_id SERIAL PRIMARY KEY,
            year VARCHAR(6) UNIQUE
        )
    ''')

    # 변경 사항 커밋
    conn.commit()

In [23]:
with conn.cursor() as cur:
    # data 테이블 생성
    cur.execute('''
        CREATE TABLE IF NOT EXISTS data (
            id SERIAL PRIMARY KEY,
            year_id INT REFERENCES years(year_id),
            location VARCHAR(50),
            value INT
        )
    ''')

    # 변경 사항 커밋
    conn.commit()

In [27]:
# 지역 리스트
locations = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주']

# 주어진 데이터 리스트
data = electric_car_last_count

# 데이터베이스에 연도와 데이터를 삽입
with conn.cursor() as cur:
    for row in data:
        year = row[0]

        # 연도 삽입
        cur.execute('''
            INSERT INTO years (year) VALUES (%s) ON CONFLICT (year) DO NOTHING RETURNING year_id
        ''', (year,))
        year_id = cur.fetchone()[0] if cur.rowcount > 0 else None

        # 각 지역의 값 삽입
        for i, location in enumerate(locations):
            value = row[i + 1]  # 연도 이후의 값이므로 row[i + 1]
            cur.execute('''
                INSERT INTO data (year_id, location, value) VALUES (%s, %s, %s)
            ''', (year_id, location, value))

    conn.commit()
